# Basalt DatasetSDK Async Demo

This notebook demonstrates the asynchronous functionality of the DatasetSDK in the Basalt Python SDK.

In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))  # Needed to make notebook work in VSCode

os.environ["BASALT_BUILD"] = "development"

from basalt import Basalt

# Initialize the SDK
basalt = Basalt(
    api_key="sk-f50...",  # Replace with your API key
    log_level="debug"  # Optional: Set log level
)

## Example 1: Asynchronously List All Datasets

This example demonstrates how to list all datasets asynchronously.

In [ ]:
async def list_datasets():
    print("Listing all datasets asynchronously...")
    err, datasets = await basalt.datasets.async_list()
    if err:
        print(f"Error listing datasets: {err}")
    else:
        print(f"Found {len(datasets)} datasets")
        for dataset in datasets:
            print(f"- {dataset.name} (slug: {dataset.slug})")
    return datasets

# Run the async function
datasets = await list_datasets()

## Example 2: Asynchronously Get a Specific Dataset

This example demonstrates how to retrieve a specific dataset by its slug.

In [ ]:
async def get_dataset(datasets):
    print("\nGetting a specific dataset asynchronously...")
    if len(datasets) > 0:
        sample_dataset = datasets[0]
        err, dataset = await basalt.datasets.async_get(sample_dataset.slug)
        if err:
            print(f"Error getting dataset: {err}")
        else:
            print(f"Retrieved dataset: {dataset.name}")
            print(f"Columns: {dataset.columns}")
            print(f"Number of rows: {len(dataset.rows) if dataset.rows else 0}")
        return sample_dataset, dataset
    else:
        print("No datasets available")
        return None, None

# Run the async function
sample_dataset, dataset = await get_dataset(datasets)

## Example 3: Asynchronously Add a Row to a Dataset

This example demonstrates how to add a new row to an existing dataset.

In [ ]:
async def add_row(sample_dataset):
    print("\nAdding a row to a dataset asynchronously...")
    if sample_dataset:
        # Create some sample values for the dataset row
        values = {column: f"Sample {column} value" for column in sample_dataset.columns}
        
        err, row, warning = await basalt.datasets.async_addRow(
            slug=sample_dataset.slug,
            values=values,
            name="Async Sample Row",
            ideal_output="Expected output for this row",
            metadata={"source": "async_example", "type": "demo"}
        )
        
        if err:
            print(f"Error adding row to dataset: {err}")
        elif warning:
            print(f"Row added with warning: {warning}")
            print(f"Row values: {row.values}")
        else:
            print(f"Row added successfully")
            print(f"Row values: {row.values}")
            print(f"Row name: {row.name}")

# Run the async function
await add_row(sample_dataset)